pytorch기반 간단한 lstm 모델입니다. 중간중간 불필요한 코드는 주석 처리 하였습니다.

## 라이브러리 로딩

In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.preprocessing import MinMaxScaler

### 예측해야될 데이터
---
- dangjin_floating : 당진수상태양광 발전량(KW)
- dangjin_warehouse : 당진자재창고태양광 발전량(KW)
- dangjin : 당진태양광 발전량(KW)
- ulsan : 울산태양광 발전량(KW)

In [ ]:
def Get_Preprocessing_Data(df, column):
    """
    모델에 입력될 데이터를 전처리하여 float형태로 변환한 후 반환하는 함수

    파라미터
    ---
    df : DataFrame
        energy 데이터
    column : str
        현재 예측할 발전소 이름

    returns 
    ---
    df : DataFrame
        전처리가 적용된 DataFrame
    """
    df = df[['time', column]]
    # 결측치는 평균 값으로 처리
    df.fillna(df.mean(), inplace=True)
    # time 컬럼을 index로 지정
    df = df.set_index('time')
    df = df[column].values.astype(float)
    return df

In [ ]:
energy = pd.read_csv('energy.csv')  # 발전소별 발전량

# 전처리 함수 적용
ulsan = Get_Preprocessing_Data(energy, 'ulsan')
dangjin_floating = Get_Preprocessing_Data(energy, 'dangjin_floating')
dangjin_warehouse = Get_Preprocessing_Data(energy, 'dangjin_warehouse')
dangjin = Get_Preprocessing_Data(energy, 'dangjin')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [ ]:
from sklearn.model_selection import train_test_split


def make_batch(input_data, sl=24):
    """
    energy 데이터를 sequence length길이에 맞춰 input형태로 변환 시 켜준다.
    그리고 train데이터 셋과 test데이터셋을 나눈다.

    파라미터 
    ---
    input_data : 
        energy 데이터
    sl : int
        sequence length 

    returns
    train_x : Tensor
        model의 학습용 input data
    train_y : Tensor
        model의 학습용 target data    
    valid_x : Tensor
        model의 검증용 input data
    valid_y : Tensor
        model의 검증용 target data    
    """
    train_x = []
    train_y = []
    L = len(input_data)
    for i in range(L-sl):
        train_seq = input_data[i:i+sl]
        train_label = input_data[i+sl:i+sl+1]
        train_x.append(train_seq)
        train_y.append(train_label)

    tensor_x, tensor_y = torch.Tensor(train_x), torch.Tensor(train_y)
    train_x, valid_x, train_y, valid_y = train_test_split(
        tensor_x, tensor_y, test_size=0.3)

    return train_x, valid_x, train_y, valid_y

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


class CustomDataset(torch.utils.data.Dataset):
    """
    mini batch 학습을 위한 customdataset
    """

    def __init__(self, tensor_x, tensor_y):
        self.x_data = tensor_x
        self.y_data = tensor_y

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx]
        return x, y

In [ ]:
# 모델 설계
class simple_lstm(nn.Module):

    def __init__(self, input_vector, sl, output_vector, num_layers):
        super().__init__()
        self.input_vector = input_vector
        self.sequence_length = sl
        self.output_vector = output_vector
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size=self.input_vector, hidden_size=self.output_vector,
                            num_layers=self.num_layers, batch_first=True)
        self.linear = nn.Sequential(
            nn.Linear(self.output_vector, 50),
            nn.Linear(50, 30),
            nn.Linear(30, 10),
            nn.Linear(10, 1)
        )

    def forward(self, x):
        output, _ = self.lstm(x)  # (hidden, cell) 데이터는 사용하지 않음
        return self.linear(output[:, -1, :])


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def training(model, EPOCHS, optimizer, criterion, train_loader, valid_loader):
    """
    model을 학습하는함수 
    검증데이터셋을 이용하여 가장 성능이 좋은 모델을 반환한다.
    """
    best_model = model
    BEST_LOSS = int(1e9)
    for i in range(EPOCHS):

        TRAIN_LOSS = 0
        VALID_LOSS = 0
        for batch_idx, batch in enumerate(train_loader):
            model.train()
            tensor_x, tensor_y = batch
            # optimizer 초기화
            optimizer.zero_grad()

            tensor_x = tensor_x.to(device)
            tensor_y = tensor_y.to(device)
            output = model(tensor_x)
            loss = criterion(output, tensor_y.view(-1, 1))

            # loss 누적
            TRAIN_LOSS += loss.item()

            loss.backward()
            optimizer.step()

        model.eval()

        for batch_idx, batch in enumerate(valid_loader):
            tensor_x, tensor_y = batch
            with torch.no_grad():
                tensor_x = tensor_x.to(device)
                tensor_y = tensor_y.to(device)
                output = model(tensor_x)
                loss = criterion(output, tensor_y.view(-1, 1))
                VALID_LOSS += loss.item()

        # best 모델 저장
        if VALID_LOSS < BEST_LOSS:
            best_model = model
            BEST_LOSS = VALID_LOSS

        if i % 100 == 0:
            print('Epoch {}, train_Loss {:.5f}, valid_Loss {:.5f}'.format(
                i, TRAIN_LOSS, VALID_LOSS))

    return best_model

In [ ]:
def inference(df, model):
    """
    학습된 모델을 이용하여 발전소별 발전량을 예측하는 함수
    """

    x_input = np.array(df[-48:])  # next value based on data of last year
    x_input = x_input.reshape((1, 48, 1))
    model_pred = []

    for i in range(672):

        x_input = torch.Tensor(x_input)
        x_input = x_input.to(device)
        predict = model(x_input).cpu().detach().numpy()

        new_input = predict.reshape((1, 1, 1))
        x_input = np.concatenate((x_input[:, -47:].cpu(), new_input), axis=1)
        model_pred.append(predict[0][0])

### dangjin_floating

In [ ]:
def train_and_inference(df):
    """
    각 발전소 별 모델 학습하고 
    성능이 가장 좋은 모델을 이용하여 추론한 결과를 반환하는 함수

    파라미터 
    ---
    df : dataframe
        발전소 별 발전량

    return
    ---
    pred : list
        학습된 모델을 이용한 예측값
    """

    # hyper parameters
    SEQUENCE_LENGTH = 48
    INPUT_VECTOR = 1
    OUTPUT_VECTOR = 100
    NUM_LAYERS = 4
    EPOCHS = 2000
    LR = 0.0001

    # 모델 선언
    lstm = simple_lstm(INPUT_VECTOR, SEQUENCE_LENGTH,
                       OUTPUT_VECTOR, NUM_LAYERS).to(device)

    optimizer = torch.optim.Adam(lstm.parameters(), lr=LR)
    criterion = nn.MSELoss()

    train_x, valid_x, train_y, valid_y = make_batch(
        df.reshape(-1, 1), SEQUENCE_LENGTH)
    trn_data = CustomDataset(train_x, train_y)
    val_data = CustomDataset(valid_x, valid_y)
    train_loader = DataLoader(trn_data, batch_size=256, shuffle=True)
    valid_loader = DataLoader(val_data, batch_size=256, shuffle=True)

    best_model = training(lstm, EPOCHS, optimizer,
                          criterion, train_loader, valid_loader)
    pred = inference(df, best_model)

    return pred

In [ ]:
dangjin_floating_pred = train_and_inference(dangjin_floating)
ulsan_pred = train_and_inference(ulsan)
dangjin_warehouse_pred = train_and_inference(dangjin_warehouse)
dangjin_pred = train_and_inference(dangjin)

Epoch 0, train_Loss 3628771.52344, valid_Loss 1528781.82031
Epoch 100, train_Loss 111320.26965, valid_Loss 52839.59729
Epoch 200, train_Loss 101892.66162, valid_Loss 52973.18640
Epoch 300, train_Loss 86359.60925, valid_Loss 55953.62732
Epoch 400, train_Loss 61679.72440, valid_Loss 62537.70178
Epoch 500, train_Loss 35888.50278, valid_Loss 70154.57422
Epoch 600, train_Loss 19431.43362, valid_Loss 76953.08704
Epoch 700, train_Loss 11116.01704, valid_Loss 81847.70544
Epoch 800, train_Loss 10459.42016, valid_Loss 85358.49255
Epoch 900, train_Loss 4990.33447, valid_Loss 84845.09924
Epoch 1000, train_Loss 1897.68170, valid_Loss 83899.68140
Epoch 1100, train_Loss 1969.11242, valid_Loss 83475.45581
Epoch 1200, train_Loss 1365.27160, valid_Loss 81939.67615
Epoch 1300, train_Loss 2567.59942, valid_Loss 82334.48157
Epoch 1400, train_Loss 1515.97293, valid_Loss 80986.61487
Epoch 1500, train_Loss 425.90983, valid_Loss 79563.20227
Epoch 1600, train_Loss 842.40405, valid_Loss 79141.02783
Epoch 1700, t

### 정답 파일 생성

In [ ]:
submission = pd.read_csv('submission.csv')

In [ ]:
submission.iloc[:24*28, 1] = dangjin_floating_pred
submission.iloc[:24*28, 2] = dangjin_warehouse_pred
submission.iloc[:24*28, 3] = dangjin_pred
submission.iloc[:24*28, 4] = ulsan_pred

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()